# Summary

Thanks for Dave to share the great PMC vectorization file: http://evexdb.org/pmresources/vec-space-models/

The vectorization part is developed from the tutorials by Francois Chollet.

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [1]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from gensim.models import KeyedVectors
import pickle
from sklearn.model_selection import GridSearchCV, train_test_split
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.optimizers import Adam



MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 200

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 Embedding

In [2]:
p_file = 'data_X'

with open(p_file, 'rb') as fin:
    data_X = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'

with open(p_file, 'rb') as fin:
    labels = pickle.load(fin)
labels = to_categorical(np.asarray(labels))

In [4]:
p_file = 'Models/vectors_small1'

with open(p_file, 'rb') as fin:
    data = pickle.load(fin)

In [5]:
p_file = 'Models/embedding_matrix_small1'

with open(p_file, 'rb') as fin:
    embedding_matrix = pickle.load(fin)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25)

In [7]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(MAX_NUM_WORDS,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# 2 LSTM

In [8]:
def model():
        
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(embedded_sequences)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = sequence_input, outputs = X)
        
    return model

In [9]:
model = model()

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 200)          4000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 768)           2304768   
_________________________________________________________________
batch_normalization_1 (Batch (None, 22, 768)           3072      
_________________________________________________________________
activation_1 (Activation)    (None, 22, 768)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 768)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 22, 512)           2623488   
__________

In [11]:
from keras.utils import plot_model
plot_model(model, to_file='LSTM.png')

<img src='LSTM.png'>

In [12]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [13]:
model.fit(X_train, Y_train, batch_size = 64, epochs=20)

Epoch 1/20
14333/14333 [==============================] - 30s 2ms/step - loss: 0.4855 - acc: 0.7714
Epoch 2/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.4201 - acc: 0.8153
Epoch 3/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3893 - acc: 0.8300
Epoch 4/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3560 - acc: 0.8483
Epoch 5/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.3225 - acc: 0.8619
Epoch 6/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.2931 - acc: 0.8777
Epoch 7/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.2682 - acc: 0.8920
Epoch 8/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.2380 - acc: 0.9030
Epoch 9/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.2260 - acc: 0.9063
Epoch 10/20
14333/14333 [==============================] - 28s 2ms/step - loss: 0.2053 - acc: 0.9177

In [14]:
model_json = model.to_json()
with open ("LSTMModel_100vectors.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModel_100vectors.h5")

In [15]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 4s 903us/step
Validation set accuracy =  0.7932189200502302


In [16]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

4778/4778 [==============================] - 4s 858us/step
             precision    recall  f1-score   support

          0       0.81      0.82      0.81      2608
          1       0.78      0.76      0.77      2170

avg / total       0.79      0.79      0.79      4778

